In [5]:
import requests
import pandas as pd
from io import StringIO
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, timedelta
import altair as alt
from vega_datasets import data
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, DatePicker
import geopandas as gpd
import json
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [6]:
#!pip install geopandas

In [7]:
def download_earthquakes(date):
    #API url
    url = "https://earthquake.usgs.gov/fdsnws/event/1/"
    #endpoint for counting records
    count_url = url + 'count'
    #endpoint for downloading records
    query_url = url + 'query'
    #query parameters
    params = {
            'format': 'geojson',
            'starttime': date.isoformat(),
            'endtime': (date + timedelta(hours = 24)).isoformat(),
            'orderby': 'time-asc',
        }
    #print(params)
    try:
        print(f'Searching for earthquakes on {date}...')
        count = requests.get(count_url, params=params, timeout=30)
        print(f'Found {json.loads(count.text)["count"]} earthquakes on {date}')
        print(f'Downloading {json.loads(count.text)["count"]} earthquakes from {date}...')
        response = requests.get(query_url, params=params, timeout=30)
        #print(response.status_code)
        if response.status_code == 200:
                    # Load data into DataFrame
                    gdf = gpd.read_file(StringIO(response.text))
                    print(f'Downloaded {len(gdf)} earthquakes from {date}')
                    return gdf
                
        else:
            print(response.text)
                
    except requests.exceptions.Timeout:
        print('Error: Timeout"')
    except Exception as e:
        print(f'Unknown Error:{str(e)}')
    
# Function to fetch earthquake data
def fetch_earthquakes(date = datetime.now().date()):
        dfs = []
        while date <= datetime.now().date():
            dfs.append(download_earthquakes(date))
            date = date.replace(year = date.year + 1)
        global df
        df = pd.concat(dfs)
        
        print(f'Successfully downloaded {len(gdf)} earthquakes')
        print(gdf.describe())
            
        df = gdf
        df['longitude'] = df.geometry.x
        df['latitude'] = df.geometry.y
        df['depth'] = df.geometry.z
        df = df[df['mag'] >= -1]
interact_manual(fetch_earthquakes, date = DatePicker(value = datetime.now().date()))

interactive(children=(DatePicker(value=datetime.date(2025, 11, 19), description='date', step=1), Button(descri…

<function __main__.fetch_earthquakes(date=datetime.date(2025, 11, 19))>

In [9]:
WIDTH = 600
HEIGHT = 300
BACKGROUND = '#444444'
DOMAIN = [df['mag'].min() - .1,df['mag'].max()]
COLOR_SCHEME = alt.Color('mag:Q').scale(scheme = 'magma', domain = DOMAIN)
LABEL_COLOR = '#888888'


PROJECTIONS = ['equalEarth', 'mercator', 'orthographic', 'naturalEarth1', 
             'equirectangular', 'stereographic', 'gnomonic', 'azimuthalEqualArea',
             'albersUsa', 'conicEqualArea', 'conicEquidistant']

SELECTORS = {}
HISTS = {}

In [10]:
brush = alt.selection_interval()
tsunami = alt.selection_point(fields = ['tsunami'], bind = 'legend')
quakes = alt.Chart(df).mark_point(filled = True).encode(
        latitude = 'latitude:Q',
        longitude = 'longitude:Q',
        size = alt.Size('sig:Q', scale = alt.Scale(domain = [0, df['sig'].max()])),
        shape = alt.Shape('tsunami:N', scale = alt.Scale(domain = [0,1], range = ['circle', 'triangle'])),
        order = alt.Order('time:T', sort = 'ascending'),
        opacity = alt.Opacity('mag:Q'),#, scale = alt.Scale(domain = [df['mag'].min(),df['mag'].max()])),
        color = alt.condition(brush, COLOR_SCHEME, alt.value('grey')),
        tooltip = ['year(time):T','place','depth', 'mag', 'sig','cdi','tsunami']
    ).add_params(
        brush,
        tsunami
    ).transform_filter(tsunami)

In [11]:
time_selection = alt.selection_interval()
SELECTORS['time'] = time_selection
time_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'yearmonth(time)',
        axis = alt.Axis(
            title = 'Date (by month)',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR
        )
    ),
    y = alt.Y(
        'count()',
        axis = alt.Axis(
            title = 'Count',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR
        )
    ),
    color = alt.condition(time_selection, COLOR_SCHEME, alt.value('grey')),
).properties(
    width = WIDTH,
    height = 25
).add_params(
    time_selection
)
HISTS['time'] = time_hist

In [12]:
mag_selection = alt.selection_interval()
SELECTORS['mag'] = mag_selection
mag_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'mag:Q',
        bin = alt.Bin(step = .1),
        axis = alt.Axis(
            title = 'Magnitude',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR
        )
    ),
    y = alt.Y(
        'count():Q',
        scale = alt.Scale(type = 'symlog'),
        axis = alt.Axis(
            title = 'Count',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR,
            values = [10,100,1000]
        )
    ),
    color = alt.condition(mag_selection, COLOR_SCHEME, alt.value('grey')),
    tooltip = ['count()']
).properties(
    width = WIDTH,
    height = 25
).add_params(
    mag_selection,
)
HISTS['mag'] = mag_hist

In [13]:
sig_selection = alt.selection_interval()
SELECTORS['sig'] = sig_selection
sig_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'sig:Q',
        bin = alt.Bin(step = 50),
        axis = alt.Axis(
            title = 'Significange',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR
        )
    ),
    y = alt.Y(
        'count():Q',
        scale = alt.Scale(type = 'symlog'),
        axis = alt.Axis(
            title = 'Count',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR,
            values = [10,100,1000]
        )
    ),
    color = alt.condition(sig_selection, COLOR_SCHEME, alt.value('grey')),
    tooltip = ['count()']
).properties(
    width = WIDTH,
    height = 25
).add_params(
    sig_selection,
)
HISTS['sig'] = sig_hist

In [14]:
cdi_selection = alt.selection_interval()
SELECTORS['cdi'] = cdi_selection
cdi_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'cdi:Q',
        bin = alt.Bin(step = 1),
        axis = alt.Axis(
            title = 'CDI',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR
        )
    ),
    y = alt.Y(
        'count():Q',
        scale = alt.Scale(type = 'symlog'),
        axis = alt.Axis(
            title = 'Count',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR,
            values = [10,100,1000]
        )
    ),
    color = alt.condition(cdi_selection, COLOR_SCHEME, alt.value('grey')),
    tooltip = ['count()']
).properties(
    width = WIDTH,
    height = 25
).add_params(
    cdi_selection,
)
HISTS['cdi'] = cdi_hist

In [15]:
depth_selection = alt.selection_interval()
SELECTORS['depth'] = depth_selection
depth_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'depth:Q',
        bin = alt.Bin(step = 10),
        axis = alt.Axis(
            title = 'Depth (km)',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR
        )
    
    ),
    y = alt.Y(
        'mag:Q',
        scale = alt.Scale(
            type = 'symlog',
            reverse = False
        ), 
        axis = alt.Axis(
            title = 'Count',
            titleColor = LABEL_COLOR,
            labelColor = LABEL_COLOR,
            values = [10,100,1000]
        )
    ),
    order = alt.Order('mag:Q', sort = 'descending'),
    color = alt.condition(depth_selection, COLOR_SCHEME, alt.value('grey')),
    tooltip = ['depth'],
).properties(
    width = WIDTH,
    height = 25
).add_params(
    depth_selection
)
HISTS['depth'] = depth_hist

In [16]:
output = alt.Chart(df).mark_rect().encode(
    x = alt.X('yearmonth(time)', 
              bin = alt.Bin(maxbins = 25), 
              axis = alt.Axis(title = 'Date', labelColor = '#888888', titleColor = '#888888')
             ),
    y = alt.Y('depth', 
              bin = alt.Bin(step = 25), 
              scale = alt.Scale(reverse = True),
              axis = alt.Axis(title = 'Depth (km)', labelColor = '#888888', titleColor = '#888888')
             ),
    color = COLOR_SCHEME,
    tooltip = alt.Tooltip(['max(mag)', 'yearmonth(time)']
                         )
).properties(
    width = 400,
    height = HEIGHT + 250
)

In [17]:
def project_earth(phi = 0, theta = 0, zoom = 2.5, projection = 'equalEarth'):
    #scale_slider = alt.param('scale', value = 180, bind = alt.binding_range(min = 100, max = 1000))
    Projection = alt.Projection(type = projection, rotate = [phi, theta], scale = zoom * 100)#, scale = scale_slider)
    
    topo = alt.topo_feature(data.world_110m.url, 'countries')
    
    earth = alt.Chart(topo).mark_geoshape(
        fill='darkgrey',
        stroke='lightgrey',
    ).properties(
        width = WIDTH,
        height = HEIGHT,
        projection = Projection
    ).add_params(
        alt.selection_interval(),
        #scale_slider
    )
    
    graticule = alt.Chart(alt.graticule()).mark_geoshape().properties(projection = Projection)
    
    earth += graticule

    return earth
    
def display_earth(phi = 0, theta = 0, zoom = 2.5, projection = 'orthographic'):
    earth = project_earth(phi, theta, zoom, projection)
    display(earth.properties(background = BACKGROUND))

#interact(display_earth, 
 #        phi = (-180,180,10), 
  #       theta = (-90,90,10), 
   ##     projection = PROJECTIONS)

In [18]:
def display_earthquakes(phi = 0, theta = 0, zoom = 2.5, projection = 'orthographic', time = True, depth = True, mag = True, sig = True, cdi = True):
    
    Projection = alt.Projection(type = projection, rotate = [phi, theta], scale = zoom * 100, translate = [WIDTH/2,HEIGHT/2])
    
    topo = alt.topo_feature(data.world_110m.url, 'countries')
    
    earth = alt.Chart(topo).mark_geoshape(
        fill='darkgrey',
        stroke='lightgrey',
    ).properties(
        width = WIDTH,
        height = HEIGHT,
        projection = Projection
    ).add_params(
        alt.selection_interval(),
        #scale_slider
    )
    
    graticule = alt.Chart(alt.graticule()).mark_geoshape().properties(projection = Projection)
    
    earth += graticule

    filters = []
    if mag:
        filters.append('mag')
    if depth:
        filters.append('depth')
    if time:
        filters.append('time')
    if sig:
        filters.append('sig')
    if cdi:
        filters.append('cdi')
        
    if filters:
        selectors = [SELECTORS[f] for f in filters]
        earth += quakes.properties(projection = Projection).transform_filter(*selectors)
    
        hists = [HISTS[f] for f in filters]
        for hist in hists:
            earth = earth & hist
    else:
        earth += quakes.properties(projection = Projection)
    earth = earth | output.transform_filter(brush).transform_filter(*selectors)
    #earthquakes = (earth + quakes).properties(background = BACKGROUND)
    display(earth.properties(background = BACKGROUND))

interact_manual(display_earthquakes, 
         phi = IntSlider(min = -180, max = 180,step = 10,continuous_update =False), 
         theta = IntSlider(min = -90, max = 90,step = 10,continuous_update =False),
         zoom = FloatSlider(min = 1, max = 10, step = .25,continuous_update =False),
         projection = PROJECTIONS,
        )
    



interactive(children=(IntSlider(value=0, continuous_update=False, description='phi', max=180, min=-180, step=1…

<function __main__.display_earthquakes(phi=0, theta=0, zoom=2.5, projection='orthographic', time=True, depth=True, mag=True, sig=True, cdi=True)>